In [1]:
#!pip install shap
#!pip install sklearn
import pandas as pd
import numpy as np
import shap
import sklearn
import seaborn as sn
import matplotlib.pyplot as plt
from numpy.linalg import eig

In [2]:
df = pd.read_csv("use_data/merged_dataset.csv")
#Fill in all NaN, needed for eigenvalues/eigenvectors
df.fillna(-1,inplace=True)
df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender
0,149002,282,885,3791,1500.0,1111,0,5.0,2.0
1,149002,282,885,3791,1500.0,1107,0,5.0,2.0
2,109881,230,629,1,1662.0,820,0,4.0,0.0
3,109881,230,629,1,1662.0,820,0,4.0,0.0
4,109881,230,629,1,1662.0,820,0,4.0,0.0
...,...,...,...,...,...,...,...,...,...
6544,73105,259,614,637,2603.0,1111,0,2.0,0.0
6545,403421,175,1181,4760,247.0,1111,0,3.0,1.0
6546,116738,186,267,1200,2276.0,1110,0,2.0,1.0
6547,116738,186,267,1200,2276.0,1101,0,2.0,1.0


In [3]:
#ingore the label column
df_array = df.to_numpy()
df_array

array([[1.49002e+05, 2.82000e+02, 8.85000e+02, ..., 0.00000e+00,
        5.00000e+00, 2.00000e+00],
       [1.49002e+05, 2.82000e+02, 8.85000e+02, ..., 0.00000e+00,
        5.00000e+00, 2.00000e+00],
       [1.09881e+05, 2.30000e+02, 6.29000e+02, ..., 0.00000e+00,
        4.00000e+00, 0.00000e+00],
       ...,
       [1.16738e+05, 1.86000e+02, 2.67000e+02, ..., 0.00000e+00,
        2.00000e+00, 1.00000e+00],
       [1.16738e+05, 1.86000e+02, 2.67000e+02, ..., 0.00000e+00,
        2.00000e+00, 1.00000e+00],
       [1.24951e+05, 1.84000e+02, 6.56000e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [4]:
#Compute mean for every dimension of the whole dataset
df_mean = df.mean().to_numpy()
df_mean

array([2.08720834e+05, 2.47627577e+02, 8.09018629e+02, 2.92348542e+03,
       3.55897710e+03, 8.82576882e+02, 3.13482974e-01, 3.18170713e+00,
       3.69369369e-01])

In [5]:
#Calculate covariance matrix for whole dataset
#The diagonal is the variance
cov_matrix = np.cov(df_array, bias=True)
cov_matrix

array([[2.16624893e+09, 2.16625616e+09, 1.60073834e+09, ...,
        1.69833404e+09, 1.69835032e+09, 1.81722453e+09],
       [2.16625616e+09, 2.16626340e+09, 1.60074357e+09, ...,
        1.69833956e+09, 1.69835585e+09, 1.81723048e+09],
       [1.60073834e+09, 1.60074357e+09, 1.18369263e+09, ...,
        1.25556732e+09, 1.25557908e+09, 1.34388401e+09],
       ...,
       [1.69833404e+09, 1.69833956e+09, 1.25556732e+09, ...,
        1.33198890e+09, 1.33200132e+09, 1.42556602e+09],
       [1.69835032e+09, 1.69835585e+09, 1.25557908e+09, ...,
        1.33200132e+09, 1.33201375e+09, 1.42557941e+09],
       [1.81722453e+09, 1.81723048e+09, 1.34388401e+09, ...,
        1.42556602e+09, 1.42557941e+09, 1.52599404e+09]])

In [6]:
#Compute eigenvectors and eigenvalues
w, v = eig(cov_matrix)
print('E-value:', w)
print('E-vector:', v)

E-value: [ 3.73652111e+13+0.00000000e+00j  4.83116539e+09+0.00000000e+00j
  2.01928936e+09+0.00000000e+00j ... -1.16967720e-72-2.66058587e-72j
 -2.12725049e-73+0.00000000e+00j -4.31751275e-84+0.00000000e+00j]
E-vector: [[ 7.61322566e-03+0.00000000e+00j -1.34708992e-03+0.00000000e+00j
  -1.55535392e-02+0.00000000e+00j ...  2.02091315e-26-2.49042705e-25j
  -2.25547709e-25+0.00000000e+00j -1.22925141e-35+0.00000000e+00j]
 [ 7.61325150e-03+0.00000000e+00j -1.34939607e-03+0.00000000e+00j
  -1.55543048e-02+0.00000000e+00j ... -1.13537366e-18+9.23788767e-19j
   2.39264595e-19+0.00000000e+00j -1.30763669e-19+0.00000000e+00j]
 [ 5.62796145e-03+0.00000000e+00j  1.30909199e-03+0.00000000e+00j
   8.40833490e-03+0.00000000e+00j ... -3.85690707e-19-1.99204671e-20j
   8.72632223e-18+0.00000000e+00j  1.11328692e-18+0.00000000e+00j]
 ...
 [ 5.97035516e-03+0.00000000e+00j -2.64081575e-03+0.00000000e+00j
   2.97838272e-03+0.00000000e+00j ...  5.11234014e-19-1.89243334e-19j
  -1.06782800e-18+0.00000000e+0

In [15]:
inds = (-w).argsort()
sortedw = sorted(w,reverse=True)
sortedv = list(v[inds])

In [16]:
#Choose top K
k = 9
sortedw[:k]

[(37365211120457.19+0j),
 (4831165387.338659+0j),
 (2019289360.090516+0j),
 (111683559.13175708+0j),
 (55695442.43207115+0j),
 (2744824.8956622514+0j),
 (1864.70070221577+0j),
 (361.2688006018666+0j),
 (0.012592534685206904+0.0045183207676350295j)]

In [26]:
#Combine two eigenvectors with the highest corresponding eigenvalues
sortedv[0]+=sortedv[1]
del sortedv[1]
del sortedw[1]
pca = np.array(sortedv[:k-1])
pca

array([[ 9.16858695e-02+0.00000000e+00j, -3.65502080e-02+0.00000000e+00j,
        -6.11027585e-03+0.00000000e+00j, ...,
        -3.44129822e-19+1.16834212e-17j,  1.17770645e-17+0.00000000e+00j,
         6.65088658e-19+0.00000000e+00j],
       [ 7.19110700e-04+0.00000000e+00j, -5.66300113e-03+0.00000000e+00j,
        -1.82799422e-02+0.00000000e+00j, ...,
         5.90077389e-19+2.65045464e-19j, -2.51960350e-18+0.00000000e+00j,
        -6.18742072e-19+0.00000000e+00j],
       [ 7.22353453e-04+0.00000000e+00j, -5.95242324e-03+0.00000000e+00j,
        -1.83760223e-02+0.00000000e+00j, ...,
         8.29329671e-19-1.12630675e-18j,  7.20558871e-18+0.00000000e+00j,
        -8.28014597e-19+0.00000000e+00j],
       ...,
       [ 2.38577743e-03+0.00000000e+00j, -2.88362699e-02+0.00000000e+00j,
        -2.90681941e-03+0.00000000e+00j, ...,
        -4.75968304e-18+2.77223077e-18j,  1.77597381e-18+0.00000000e+00j,
         4.15964166e-19+0.00000000e+00j],
       [ 2.53202004e-03+0.00000000e+00j, -3.

In [27]:
pca.shape

(8, 6549)